In [78]:
import pandas as pd
import re
import numpy as np
from collections import Counter
import json

In [2]:
df = pd.read_csv('fwb-australia-topic-prediction.csv')

In [19]:
df.preprocessed_list_with_bi_tri[0]

"['form' 'compani' 'famili' 'much' 'land' 'hand' 'farm' 'children' 'part'\n 'succession_plan' 'final' 'decis' 'inevit' 'differ' 'everi' 'famili'\n 'work' 'issu' 'number' 'children' 'number' 'farm' 'children' 'amount'\n 'farm' 'debt' 'provis' 'made' 'parent' 'retir' 'coupl' 'farm' 'southern'\n 'success' 'plan' 'give' 'farm' 'son' 'run' 'properti' 'still' 'retain'\n 'control' 'land' 'start' 'think' 'plan' 'earli' 'prompt' 'main' 'account'\n 'suggest' 'could' 'form' 'compani' 'reduc' 'taxat' 'payment' 'would'\n 'turn' 'give' 'abil' 'fund' 'farm' 'superannu' 'seek' 'profession'\n 'advic' 'famili' 'compani' 'husband' 'wife' 'director' 'famili' 'trust'\n 'also' 'land' 'bought' 'either' 'compani' 'trust' 'name' 'requir' 'work'\n 'presumpt' 'least' 'son' 'would' 'return' 'home' 'sought' 'advic' 'earli'\n 'plan' 'allow' 'purchas' 'land' 'right' 'entiti' 'expand' 'busi' 'parent'\n 'retain' 'control' 'busi' 'time' 'offici' 'hand' 'son' 'like' 'will'\n 'happen' 'land' 'own' 'trust' 'compani' 'land

In [30]:
# Convert the list of words within a string 
# to a list of words
df.preprocessed_list_with_bi_tri = df.preprocessed_list_with_bi_tri.apply(lambda x: re.findall(r"'(\w+)'", x))

In [35]:
df

,text,region_of_origin,publication_date,publisher_name,preprocessed_sentence,preprocessed_list_with_bi_tri,preprocessed_len,topics,corrected_topics
0,"Forming a company key for one family HOW, wh...",AUSNZ AUSTR,1120089600000,West Australian Newspapers Limited,"['forming', 'company', 'family', 'much', 'land...","[form, compani, famili, much, land, hand, farm...",320,Topic 20,topic 20
1,"There are many different types of homebuyers, ...",AUSNZ AUSTR,1493424000000,Fairfax Media Management Pty Limited,"['many', 'different', 'types', 'homebuyers', '...","[mani, differ, type, homebuy, understand, help...",68,Topic 40,topic 40
2,Off-the-plan contracts review The NSW Governme...,AUSNZ AUSTR,1517616000000,Fairfax Media Management Pty Limited,"['plan', 'contracts', 'review', 'government', ...","[plan, contract, review, govern, releas, discu...",59,Topic 36,topic 36
3,'Super must rise to 12%': MP Whitlam MP Stephe...,AUSNZ AUSTR,1570579200000,Fairfax Media Management Pty Limited,"['super', 'must', 'rise', 'whitlam', 'stephen'...","[super, must, rise, whitlam, stephen_jon, say,...",130,Topic 28,topic 28
4,Where there's a will TODAY I'd like to talk ab...,AUSNZ AUSTR,1558742400000,Fairfax Media Management Pty Limited,"['today', 'like', 'talk', 'death', 'attention'...","[today, like, talk, death, attent, heaven_sak,...",333,Topic 46,topic 59
...,...,...,...,...,...,...,...,...,...
119942,Super choice brings costs ONE in two small b...,AUSNZ AUSTR,1130889600000,Nationwide News Pty Ltd.,"['super', 'choice', 'brings', 'costs', 'small'...","[super, choic, bring, cost, small, busi, owner...",42,Topic 47,topic 45
119943,"Home loans guide NO doc, low doc, combinatio...",AUSNZ AUSTR,1155686400000,Nationwide News Pty Ltd.,"['home', 'loans', 'guide', 'combination', 'spl...","[home, loan, guid, combin, split, honeymoon, s...",82,Topic 50,topic 50
119944,Taxing times for families TAXPAYERS having the...,AUSNZ AUSTR,1216771200000,Nationwide News Pty. Ltd.,"['taxing', 'times', 'families', 'taxpayers', '...","[tax, time, famili, taxpay, return, prepar, aw...",112,Topic 36,topic 36
119945,Legal papers vital Legal papers vital W...,AUSNZ AUSTR,1188950400000,Nationwide News Pty Ltd.,"['legal', 'papers', 'vital', 'legal', 'papers'...","[legal, paper, vital, legal, paper, vital, cen...",158,Topic 43,topic 40


In [36]:
# Convert the timestamp column to datetime
df['publication_date'] = pd.to_datetime(df['publication_date'], unit='ms')

In [38]:
df['year'] = df.publication_date.dt.year

# Convert it to a json format

## For 2022

In [136]:
def get_filtered_data_on_year(year):
    df_year = df[df.year == year]
    list_of_year_words = np.concatenate(df_year.preprocessed_list_with_bi_tri.values).tolist()
    # Calculate the frequencies of each word
    word_frequencies_year = Counter(list_of_year_words)
    # Convert the frequencies to a dataframe
    frequencies_df_year = pd.DataFrame.from_dict(word_frequencies_year, orient='index', columns=['Frequency'])
    # Sort the dataframe by frequency in descending order
    frequencies_df_year = frequencies_df_year.sort_values('Frequency', ascending=False)
    frequencies_df_year = frequencies_df_year.iloc[0:60]
    return frequencies_df_year, df_year

In [134]:
def get_json_data(frequencies_df):
    json_data_year = []
    for i,r in frequencies_df.iterrows():
        json_data = {"word": '', "frequency": 0}
        json_data.update({"word": i, "frequency": int(r[0])})
        json_data_year.append(json_data)
    # Serialize the JSON data
    json_string = json.dumps(json_data_year, ensure_ascii=False)

    # Print the JSON string
    return json_string


In [137]:
frequencies_df_2022, df_2022 = get_filtered_data_on_year(year=2022)

In [138]:
get_json_data(frequencies_df_2022)

'[{"word": "year", "frequency": 18403}, {"word": "cent", "frequency": 15859}, {"word": "rate", "frequency": 13055}, {"word": "said", "frequency": 12952}, {"word": "super", "frequency": 12772}, {"word": "fund", "frequency": 12598}, {"word": "invest", "frequency": 11414}, {"word": "home", "frequency": 10512}, {"word": "would", "frequency": 10306}, {"word": "financi", "frequency": 9084}, {"word": "say", "frequency": 8558}, {"word": "price", "frequency": 8321}, {"word": "market", "frequency": 7732}, {"word": "retir", "frequency": 7651}, {"word": "time", "frequency": 7448}, {"word": "pension", "frequency": 7305}, {"word": "incom", "frequency": 7078}, {"word": "also", "frequency": 6809}, {"word": "make", "frequency": 6634}, {"word": "interest", "frequency": 6620}, {"word": "superannu", "frequency": 6527}, {"word": "servic", "frequency": 6502}, {"word": "properti", "frequency": 6474}, {"word": "need", "frequency": 6425}, {"word": "cost", "frequency": 6419}, {"word": "peopl", "frequency": 6333

## For 2019

In [139]:
frequencies_df_2019, df_2019 = get_filtered_data_on_year(year=2019)

In [131]:
get_json_data(frequencies_df_2019)

'[{"word": "said", "frequency": 16608}, {"word": "year", "frequency": 14500}, {"word": "cent", "frequency": 12152}, {"word": "home", "frequency": 11402}, {"word": "servic", "frequency": 10913}, {"word": "fund", "frequency": 9979}, {"word": "would", "frequency": 9320}, {"word": "retir", "frequency": 8825}, {"word": "rate", "frequency": 8499}, {"word": "invest", "frequency": 8484}, {"word": "bank", "frequency": 8425}, {"word": "super", "frequency": 8408}, {"word": "financi", "frequency": 8222}, {"word": "properti", "frequency": 7260}, {"word": "pension", "frequency": 7079}, {"word": "peopl", "frequency": 6954}, {"word": "time", "frequency": 6627}, {"word": "loan", "frequency": 6601}, {"word": "also", "frequency": 6204}, {"word": "make", "frequency": 6178}, {"word": "incom", "frequency": 6013}, {"word": "market", "frequency": 5943}, {"word": "australian", "frequency": 5934}, {"word": "govern", "frequency": 5911}, {"word": "need", "frequency": 5732}, {"word": "superannu", "frequency": 5348

# For the bar chart

## For 2022

In [151]:
topic_names_inc_exc = pd.read_excel('../../../../australia/Financial_wellbeing_topic_names.xlsx', sheet_name='in')
# Retain the appropriate topics only
imp_topics_df = topic_names_inc_exc
imp_topics_df['Topic Number'] = imp_topics_df['Topic Number'].apply(lambda x: 'topic '+str(x))
topics_renamed = dict(zip(imp_topics_df['Topic Number'].values, imp_topics_df['Topic Name (created by research team)'].values))
df_2022['renamed_topics'] = df_2022.corrected_topics.map(topics_renamed)
df_2022

c:\Users\E116189\RMIT\Topic-modelling\framework\tpm\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [154]:
df_2022_topic_counts = df_2022.renamed_topics.value_counts()

In [ ]:
#  { date: new Date("2021-03-01"), value: 15, label: 'Topic 2' }],

In [179]:
dates = []

for num, (i, r) in enumerate(df_2022_topic_counts.iteritems()):
    dates.append({'date: ' + f'new Date("2021-0{num+3}-01"), value: {r}, label: "{i}"'})

In [180]:
dates

[{'date: new Date("2021-03-01"), value: 825, label: "Billionaires"'},
 {'date: new Date("2021-04-01"), value: 442, label: "Criminal law"'},
 {'date: new Date("2021-05-01"), value: 373, label: "Superannuation"'},
 {'date: new Date("2021-06-01"), value: 301, label: "Pubilc statements (see 38) "'},
 {'date: new Date("2021-07-01"), value: 257, label: "Social security and government benefits"'},
 {'date: new Date("2021-08-01"), value: 244, label: "Life insurance"'},
 {'date: new Date("2021-09-01"), value: 233, label: "Food and cooking"'},
 {'date: new Date("2021-010-01"), value: 190, label: "Property market"'},
 {'date: new Date("2021-011-01"), value: 152, label: "Food and nutrition"'},
 {'date: new Date("2021-012-01"), value: 151, label: "Start ups"'},
 {'date: new Date("2021-013-01"), value: 150, label: "Education and training"'},
 {'date: new Date("2021-014-01"), value: 124, label: "Trusts and lending"'},
 {'date: new Date("2021-015-01"), value: 107, label: "Index funds"'},
 {'date: new 